In [1]:
from adam_impact_study.types import (
    ImpactorOrbits,
    Observations,
    ResultsTiming,
    WindowResult,
    ImpactorResultSummary,
)
import ray
from adam_impact_study.analysis.main import (
    run_all_analysis,
    collect_all_results,
    summarize_impact_study_results,
    make_analysis_plots,
)
from adam_impact_study.analysis.collect import (
    collect_all_window_results_new,
    create_missing_window_results,
    collect_all_observations,
)

In [2]:
ray.shutdown()

In [3]:
run_dir = "../march_run/results"
out_dir = "../march_analysis/"

In [4]:
# impactor_orbits, observations, results_timing, window_results = collect_all_results(run_dir, max_processes=10)

INFO:adam_impact_study.analysis.collect:Collecting impactor orbits for ../march_run/results
INFO:adam_impact_study.analysis.collect:Loading 30000 impactor orbits files
INFO:adam_core.ray_cluster:Ray is not initialized. Initializing...
INFO:adam_core.ray_cluster:Attempting to connect to existing ray cluster...
INFO:adam_core.ray_cluster:Could not connect to existing ray cluster.
INFO:adam_core.ray_cluster:Attempting ray with 10 cpus and None bytes.
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/subprocess.py:1883: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = _fork_exec(
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/subprocess.py:1883: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = _fork_exec(
2025-04-08 15:47

In [5]:
# impactor_orbits.to_parquet(
#     "../march_analysis/impactor_orbits.parquet"
# )
# observations.to_parquet(
#     "../march_analysis/observations.parquet"
# )
# results_timing.to_parquet(
#     "../march_analysis/results_timing.parquet"
# )
# window_results.to_parquet(
#     "../march_analysis/window_results.parquet"
# )


In [6]:
impactor_orbits = ImpactorOrbits.from_parquet(
    "../march_analysis/impactor_orbits.parquet"
)
observations = Observations.from_parquet(
    "../march_analysis/observations.parquet"
)
results_timing = ResultsTiming.from_parquet(
    "../march_analysis/results_timing.parquet"
)
window_results = WindowResult.from_parquet(
    "../march_analysis/window_results.parquet"
)


In [ ]:
summary_results = summarize_impact_study_results(
    impactor_orbits, observations, results_timing, window_results, out_dir, max_processes=10
)
summary_results.to_parquet(
    "../march_analysis/summary_results.parquet"
)


In [7]:
summary_results = ImpactorResultSummary.from_parquet(
    "../march_analysis/summary_results.parquet"
)



In [ ]:
make_analysis_plots(summary_results, window_results, out_dir)